# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error when working with explicit data and is cross-entropy loss when working with implicit data

In [1]:
const name = "UserItemBiases"
const residual_alphas = []
const implicit = false;

In [2]:
import Statistics: mean
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb")

read_alpha (generic function with 2 methods)

In [3]:
# TODO support residualization
const training = get_split("training", implicit = implicit)
const validation = get_split("validation", implicit = implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge

In [4]:
function make_prediction(user, items, u, a, implicit)
    implicit ? implicit_make_prediction(user, items, u, a) :
    explicit_make_prediction(user, items, u, a)
end

function train_model(training, stop_criteria, implicit, λ)
    implicit ? implicit_train_model(training, stop_criteria, λ) :
    explicit_train_model(training, stop_criteria, λ)
end;

## ALS for Explicit data
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [5]:
function explicit_make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        @inbounds r[i] = u[users[i]] + a[items[i]]
    end
    r
end;

In [6]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    @inbounds for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function explicit_update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)
    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [7]:
function explicit_train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training"), log(λ_wu)) .*
        expdecay(get_counts("training"; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        explicit_update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        explicit_update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

## ALS for implicit data
* TODO

In [8]:
function implicit_make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        r[i] = softmax(u[users[i]] .+ a)[items[i]]
    end
    r
end;

In [98]:
function implicit_update_user(u, a, w, λ_u, y)
    lossfn(x) = loss(softmax(x .+ a), y, w, true) + λ_u * (x - u)^2 # TODO implicit
    Optim.minimizer(Optim.optimize(
        x -> lossfn(x...),
        fill(0.0f0, 1),
        Optim.BFGS(),
        #autodiff = :forward,
    ))[1]
end;

In [95]:
function implicit_update_users!(users, items, ratings, weights, u, a, λ_u)
    R = sparse(items, users, ratings, length(a), length(u))
    W = sparse(items, users, weights, length(a), length(u))
    u_mean = mean(u)
    @tprogress Threads.@threads for i = 1:length(u)
        y = collect(R[:, i])
        w = collect(W[:, i])
        u[i] = implicit_update_user(u_mean, a, w, λ_u, y)
    end
    u
end

implicit_update_users! (generic function with 1 method)

In [96]:
u = rand(Float32, num_users())
a = rand(Float32, num_items());

In [ ]:
implicit_update_users!(
    training.user,    
    training.item,
    fill(1.0f0, length(training.rating)),
    get_weights("training", "inverse"),
    u,
    a,
    1f0,
)

Progress:   0%|▏                                        |  ETA: 2:25:11m

In [10]:
function implicit_train_model(training, stop_criteria, λ)
    return
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training"), log(λ_wu)) .*
        expdecay(get_counts("training"; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), maximum(users))
    a = zeros(eltype(λ_a), maximum(items))

    while !stop!(stop_criteria, [u, a])
        implicit_update_users!(users, items, ratings, weights, u, a, λ_u)
        implicit_update_users!(items, users, ratings, weights, a, u, λ_a)
    end
    u, a
end;

## Training

In [11]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, implicit, λ)

    r = make_prediction(validation.user, validation.item, u, a, implicit)
    residualized_loss(r, residual_alphas, implicit)
end;

In [12]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.BFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220523 02:38:13 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:01 ( 0.35 μs/it)39m
Progress: 100%|███████████████████████████| Time: 0:00:00 (35.19 ns/it)
[ Info: 20220523 02:38:39 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0001966,0.001265263,4.9162406e-7,-0.0022688252,-0.00932882)]


Iter     Function value   Gradient norm 
     0     1.820268e+00     5.538846e-02
 * Current step size: 1.0
 * time: 0.02761697769165039
 * g(x): Float32[-0.005109214, -1.1346601f-6, 0.019632269, 0.05538846]
 * ~inv(H): Float32[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220523 02:38:42 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0051223,1.0051223,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000012,0.0,1.0000012,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9805592,0.0,0.0,0.9805592,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9461175,0.0,0.0,0.0,0.9461175)]
[ Info: 20220523 02:38:58 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0009537,0.0018737506,1.1896808e-6,-0.0039003587,-0.016039185)]
[ Info: 20220523 02:38:58 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0049009,1.0049009,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(valid

     1     1.818367e+00     1.615661e-02
 * Current step size: 0.956882
 * time: 34.644192934036255
 * g(x): Float32[-0.00087074545, -1.928879f-6, 0.01615661, -0.004676832]
 * ~inv(H): Float32[1.002769 3.2100588f-6 -0.02979592 -0.008909475; 3.2100586f-6 1.0 -1.6588934f-5 -3.0112096f-5; -0.029795919 -1.6588934f-5 1.1880993 0.24190427; -0.008909461 -3.0112094f-5 0.24190421 0.8677505]
 * x: Float32[0.004888915, 1.0857359f-6, -0.018785765, -0.053000223]


[ Info: 20220523 02:39:14 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.006221,1.006221,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000031,0.0,1.0000031,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9637956,0.0,0.0,0.9637956,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9485147,0.0,0.0,0.0,0.9485147)]
[ Info: 20220523 02:39:30 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0009806,0.0018881458,1.281918e-6,-0.0039233626,-0.016296247)]
[ Info: 20220523 02:39:30 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0115192,1.0115192,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validati

...

     7     1.814313e+00     2.896635e-04
 * Current step size: 237.72826
 * time: 343.20571780204773
 * g(x): Float32[-2.82641f-5, 3.1898708f-5, 0.00020390419, 0.0002896635]
 * ~inv(H): Float32[722.5036 39.4396 48.93262 49.49653; 39.4396 3.1559913 2.7142951 2.694774; 48.932617 2.714295 17.317152 0.61425453; 49.496525 2.6947737 0.61425364 4.332232]
 * x: Float32[-0.270265, -0.016278984, -0.4311861, -0.013954893]


[ Info: 20220523 02:44:23 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7592554,0.7592554,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98353803,0.0,0.98353803,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6481724,0.0,0.0,0.6481724,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9860758,0.0,0.0,0.0,0.9860758)]
[ Info: 20220523 02:44:39 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0012355,0.0020178615,7.5714242e-6,-0.0036487915,-0.01983415)]
[ Info: 20220523 02:44:39 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.759415,0.759415,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(valida

     8     1.814313e+00     3.265770e-05
 * Current step size: 0.959205
 * time: 376.09922885894775
 * g(x): Float32[-5.2927993f-7, 3.26577f-5, -2.6372782f-6, -1.0431475f-5]
 * ~inv(H): Float32[726.86676 40.132885 49.449295 49.712696; 40.13289 3.2252777 2.9441333 2.7137015; 49.44929 2.944133 16.844332 0.6955914; 49.712692 2.7137012 0.6955905 4.3371243]
 * x: Float32[-0.2752069, -0.01658591, -0.4335002, -0.014019278]


[ Info: 20220523 02:44:56 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7592047,0.7592047,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98350364,0.0,0.98350364,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6482243,0.0,0.0,0.6482243,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98606354,0.0,0.0,0.0,0.98606354)]
[ Info: 20220523 02:45:12 regression coefficients: ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.001235,0.0020178044,7.569275e-6,-0.0036491402,-0.019833054)]
[ Info: 20220523 02:45:12 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.75836414,0.75836414,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(v

     9     1.814313e+00     6.797120e-05
 * Current step size: 8.410485
 * time: 441.56371879577637
 * g(x): Float32[-1.0208426f-5, 3.272439f-5, 1.344871f-5, 6.79712f-5]
 * ~inv(H): Float32[722.35156 104.015 49.533615 49.217503; 104.015 25.512653 7.2309823 6.1849666; 49.533615 7.2309823 16.875181 0.68392307; 49.2175 6.1849666 0.68392223 4.296307]
 * x: Float32[-0.27753612, -0.01698975, -0.4336541, -0.0141474055]


In [13]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220523 02:46:01 The optimal λ is Float32[0.75764817, 0.98315376, 0.64813644, 0.9859522], found in 28 function calls


In [18]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, implicit, λ);

[ Info: 20220523 04:38:29 training model with parameters Float32[0.75764817, 0.98315376, 0.64813644, 0.9859522]


In [19]:
#[ Info: 20220522 21:58:57 The optimal λ is Float32[0.74455047, 0.00085658807, 0.6458768, 0.9851336], found in 22 function calls

In [41]:
validation_mse(Optim.minimizer(res))

[ Info: 20220523 04:47:08 training model with parameters Float32[0.75764817, 0.98315376, 0.64813644, 0.9859522]
[ Info: 20220523 04:47:13 regression coefficients: Float32[0.99971974]


1.8145473f0

## Inference

In [21]:
model(users, items) = make_prediction(users, items, u, a, implicit)
write_alpha(model, residual_alphas, implicit);

[ Info: 20220523 04:39:10 regression coefficients: Float32[0.99971974]
[ Info: 20220523 04:39:11 validation loss: 1.8145473
[ Info: 20220523 04:39:14 training loss: 1.6736505


In [28]:
a = read_alpha("UserItemBiases", "validation").rating;

In [37]:
residualized_loss(a, residual_alphas, implicit)

[ Info: 20220523 04:43:45 regression coefficients: Float32[0.99971974]


1.8145473f0

In [34]:
loss(a, validation.rating, get_weights("validation", "inverse"), implicit)

1.8144825f0

In [17]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));

LoadError: UndefVarError: u not defined